In [1]:
import pandas as pd
from collections import Counter
import ast
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils.genres import *
from src.utils.genres import name_plot, count_genre_over_years, top_years_for_genre, bottom_years_for_genre, count_genres_by_year, calc_genre_differences, calc_genre_growth, calc_proportion_growth
import warnings
import csv
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import ne_chunk
from nltk.sentiment import SentimentIntensityAnalyzer
from src.utils import google_trends as gt
from data.Cleaning import final_df
warnings.simplefilter("ignore")

               WarName       StateName  Side  StartYear  EndYear  Outcome  \
0   Franco-Spanish War           Spain     2       1823     1823        2   
1   Franco-Spanish War          France     1       1823     1823        1   
2  First Russo-Turkish  Ottoman Empire     2       1828     1829        2   
3  First Russo-Turkish          Russia     1       1828     1829        1   
4     Mexican-American          Mexico     2       1846     1847        2   

   BatDeath  
0       600  
1       400  
2     80000  
3     50000  
4      6000  
Index(['WarName', 'Side', 'StartYear', 'EndYear', 'StateName', 'BatDeath'], dtype='object')
                                               WarName  \
0                                        Anglo-Persian   
1                                        Anglo-Persian   
2                                         Arab-Israeli   
3                                         Arab-Israeli   
4                                                Assam   
..           

In [ ]:
# Import data
data = pd.read_csv("data/filtered_movie_data.csv")
df = pd.DataFrame(data)